In [1]:
import pandas as pd

In [16]:
df_movies = pd.read_csv("Movies_all.csv")
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86493 entries, 0 to 86492
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    86493 non-null  int64  
 1   title                 86493 non-null  object 
 2   vote_average          86493 non-null  float64
 3   vote_count            86493 non-null  int64  
 4   status                86493 non-null  object 
 5   release_date          86427 non-null  object 
 6   revenue               86493 non-null  int64  
 7   runtime               86493 non-null  int64  
 8   adult                 86493 non-null  bool   
 9   backdrop_path         70442 non-null  object 
 10  budget                86493 non-null  int64  
 11  homepage              17774 non-null  object 
 12  imdb_id               86467 non-null  object 
 13  original_language     86493 non-null  object 
 14  original_title        86493 non-null  object 
 15  overview           

In [3]:
df_crew = pd.read_csv(
    "Crew_dataset.gz",
    sep="\t",
    na_values="\\N",
    usecols=["tconst", "directors"]
)

In [4]:
df_name = pd.read_csv(
    "Basic_names_crew_dataset.gz",
    sep="\t",
    na_values="\\N",
    usecols=["nconst", "primaryName"]
)

In [5]:
df_principals = pd.read_csv(
    "title.principals.tsv.gz",
    sep="\t",
    na_values="\\N",
    usecols=["tconst", "ordering", "nconst", "category"]
)

In [6]:
df_crew = df_crew.dropna(subset=["directors"]).copy()
df_crew["directors"] = df_crew["directors"].str.split(",")

# Exploding records
df_directors = df_crew.explode("directors").rename(columns={"directors": "nconst"})

# Merge
df_directors = df_directors.merge(df_name, on="nconst", how="inner")

# Combing, as multiple outputs
df_directors_grouped = (
    df_directors
    .groupby("tconst", as_index=False)
    .agg({"primaryName": lambda x: ", ".join(x.dropna().unique())})
    .rename(columns={"primaryName": "Directors"})
)

In [7]:
# Filter title.principals for rows where category is "actor" or "actress"
df_cast = df_principals[
    df_principals["category"].isin(["actor", "actress"])
].copy()

df_cast = df_cast.merge(df_name, on="nconst", how="inner")

df_cast_grouped = (
    df_cast
    .groupby("tconst", as_index=False)
    .agg({"primaryName": lambda x: ", ".join(x.dropna().unique())})
    .rename(columns={"primaryName": "Cast"})
)

In [8]:
df_cast["ordering"] = pd.to_numeric(df_cast["ordering"], errors="coerce")
df_cast.sort_values(["tconst", "ordering"], inplace=True)

# First 3 members matter, validation of a star (who is mentioned first) of the movie
df_top3 = df_cast.groupby("tconst").head(3).copy()

df_star_grouped = (
    df_top3
    .groupby("tconst", as_index=False)
    .agg({"primaryName": lambda x: ", ".join(x.dropna().unique())})
    .rename(columns={"primaryName": "StarActors"})
)

In [9]:
df_dir_cast = df_directors_grouped.merge(df_cast_grouped, on="tconst", how="inner")
df_dir_cast_star = df_dir_cast.merge(df_star_grouped, on="tconst", how="inner")

In [10]:
df_final = df_movies.merge(
    df_dir_cast_star,
    left_on="imdb_id",
    right_on="tconst",
    how="inner"
)

In [11]:
columns_to_drop = ["tconst", "nconst", "category", "ordering"]
for col in columns_to_drop:
    if col in df_final.columns:
        df_final.drop(columns=[col], inplace=True)

In [15]:
df_final.head()
df_final.to_csv("Movies_with_cast.csv", index=False)


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,tagline,genres,production_companies,production_countries,spoken_languages,keywords,movieId,Directors,Cast,StarActors
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc...",79132,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...","Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,...",109487,Christopher Nolan,"Matthew McConaughey, Anne Hathaway, Jessica Ch...","Matthew McConaughey, Anne Hathaway, Jessica Ch..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f...",58559,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...","Christian Bale, Heath Ledger, Aaron Eckhart"
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ...",72998,James Cameron,"Sam Worthington, Zoe Saldaña, Sigourney Weaver...","Sam Worthington, Zoe Saldaña, Sigourney Weaver"
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com...",89745,Joss Whedon,"Robert Downey Jr., Chris Evans, Scarlett Johan...","Robert Downey Jr., Chris Evans"
